# Lecture 5 Advanced Regression Models

In [1]:
# import necessary libraries and specify that graphs should be plotted inline
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline 

# warnings reported for function updates, ignore them
import warnings
warnings.filterwarnings('ignore')

In this practice, we implement three advanced regression models: Polynomial Regression, Ridge Regression, and LASSO Regression. 

**Note: For all the model-relevant syntax, you can google the syntax (in bold) and get its manual.**

## 1. Polynomial Regression
### 1.1 Polynomial Regression Basics
In a polynomial regression, the relationship between $y$ and $x$ is modeled as "$k$<sup>th</sup> degree polynomial" in $x$. 

For $k$<sup>th</sup> degree polynomial, the model is shown as:

<center>$y = \beta_0 + \beta_1 x + \beta_2 x^2 + ... + \beta_k x^k + \varepsilon$</center>

**Polynomial with Multiple Variables**

Note that when we have multiple variables, say, x1 and x2, the polynomials would be all potential combinations of x1 and x2. For example, a model with 2nd degree polynomial for (x1, x2) would be:


<center>$y = \beta_0 + \beta_1 x_1 + \beta_2 x_1^2 + \beta_3 x_2 + \beta_4 x_2^2 + \beta_5 x_1 x_2 + \varepsilon$</center>

It is obvious that when the degree is higher, and when we have more variables, writing out the polynomials would be extremely tedious. (Fortunately, we do not need to generate the polynomials ourselves in Python.)


### Data Loading and Splitting
As multiple variables will be created in polynomial regression, we use a single input variable $x$ for simplicity. We will use the same dataset in the previous lecture, "house.csv". The dependent variable is **'TOTAL_VALUE'**. The independent variable is **'LOT_SQFT'**.

**Practice:** 
- Let dependent variable be **'TOTAL_VALUE'**. Let independent variable be **'LOT_SQFT'**. 
    - Note: Use Series.to_frame() method to convert Series to DataFrame (i.e., 1D to 2D)
- Split the data into 70% training and 30% test set. Set seed (random_state) = 42.
- Check sample size of training and test set. 

In [2]:
# Data Loading
house = pd.read_csv('house.csv')

# Define X and Y Below, print the shape of house_X
house_X = house['LOT_SQFT']
house_y = house['TOTAL_VALUE']
print(house_X.shape)

# Note that house_X is a 1D array, which cannot fit in sklearn models as x.
house_X = house_X.to_frame()
print(house_X.shape)

# Alternative -- Define a 2-D structure, 
#                which is often required for machine learning models 
#                that expect input features to be in DataFrame format.
house_X = house[['LOT_SQFT']]
print(house_X.shape)

(5802,)
(5802, 1)
(5802, 1)


In [3]:
# Data Splitting, check sample
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(house_X, house_y, 
                                                    test_size = 0.3, random_state = 42)

### 1.2 Polynomial Regression with Scikit-Learn

Estimating polynomial regression takes one extra step compared to linear regression. The reason is that generating polynomials is tedious, and we need a specific function to complete this step. Thus, the first step is to generate polynomials, and the second step is to run the linear regression.

Specifically, the two steps are: 

- First, specify the degree of polynomial regression (i.e., speicfy $k$). Generate variables based on the specified degree. This step is done by creating a new polynomial feature object using syntax: 
<br> <center><span style="font-family:Calibri"> **sklearn.preprocessing.PolynomialFeatures(degree)** </span></center>
    - Use .fit_transform(X) method the get the transfered features.
    - You need to specify the degree (i.e., degree = xxx) before training.

- Second, run a linear regression based on the features generated in the first step. This step is done by: 
<br> <center><span style="font-family:Calibri"> **sklearn.linear_model.LinearRegression()** </span></center>
    - Recall that we have learned its features & functions: .fit, .predict, .score, .coef_, .intercept_

**Practice**
- Assume $k=3$. Run polynomial regression. (Hint: first create the polynomial features, then run the linear regression).
- Obtain and report the mse for test set. (Hint: You need to generate polynomial features for test set to do predictions.)
- Obtain and report the coefficient estimates (include and specify intercept).

In [4]:
# S1. Obtain Polynomial Features 
from sklearn.preprocessing import PolynomialFeatures

## S1.1 Define polynomial generation function and set the degree. Change x to x^0, x, x^2, x^3

poly = PolynomialFeatures(degree = 3)

In [5]:
# S1.2 Obtain the variables: Which set?

X_train_poly = poly.fit_transform(X_train)
X_train_poly.shape, X_train.shape

((4061, 4), (4061, 1))

In [6]:
poly.get_feature_names_out()        # obtain the names of the new features created

array(['1', 'LOT_SQFT', 'LOT_SQFT^2', 'LOT_SQFT^3'], dtype=object)

In [7]:
# S2. Run Linear Regression
from sklearn.linear_model import LinearRegression

# S2.1, define the linear regression function (plug in x^0, x, x^2, x^3, then obtain betas)
lr = LinearRegression()

## S2.2, train the model
lr.fit(X_train_poly,y_train)


LinearRegression()

In [8]:
# S3. Predict and calculate error
X_test_poly = poly.fit_transform(X_test)

y_pred_test = lr.predict(X_test_poly)

## Calculate mse
e = y_test - y_pred_test
print(np.sum(e**2) / y_pred_test.shape[0])

## Calculate mse - alternative 
print(np.mean(e**2))

## Calculate mse - alternative 
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, y_pred_test))


7045.946151190245
7045.946151190245
7045.946151190245


In [9]:
# S4. Report R^2 estimates
from sklearn.metrics import r2_score
print(r2_score(y_test, y_pred_test))

# alternative
print(lr.score(X_test_poly, y_test))


0.3235362995692006
0.3235362995692006


### Make Pipelines (Technical Pre-requisite to Obtain Optimal Degree)

We need to define two functions that comes in a specific order to complete the previous task. In practice, it is recommended to use a "pipeline" to automate this process.

To put two or multiple steps (e.g., PolynomialFeatures and LinearRegression) together with a specific sequence, we create a pipeline object using syntax:
<br> <center><span style="font-family:Calibri"> **sklearn.pipeline.make_pipeline()** </span></center>
- The inputs are the models used in the process. The order of input should be the order of your models/objects.
- The pipeline object will replace your original model for estimation. You can imagine that make_pipeline is putting your models in a bucket following a specific order.

Similar to other models, we can still use  .fit, .fit_transform, .score, .predict and so on.

Specific to make_pipeline(), we can use **.named_steps** to obtain the models inside the bucket. If the model is trained, then all the necessary information can be accessed as well (e.g., coefficients, intercept, etc.) To specify which model you want to look into, use ['Python_Defined_Model_Name']

**Practice: Using make_pipeline for simple polynomial regression**

Use make_pipeline to run the polynomial regression with degree = 3. 
- Define the two steps first. Then put them in a pipeline.
- Train the model using the pipeline you created.
- Obtain the mse for test set.
- Compare the process with the previous practice, what are the differences in the progress?

In [10]:
### In below, use make_pipeline for practice
from sklearn.preprocessing  import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

# S1. Define models - two models, polynomial transfer first, then linear regression 
poly_first = PolynomialFeatures(degree = 3)
lr_second = LinearRegression()


# S2. Apply polynomial regression in pipeline
poly_pipe = make_pipeline(poly_first, lr_second)

poly_pipe.fit(X_train, y_train)


# S3. Predict and get mse
y_pred_auto = poly_pipe.predict(X_test)

mean_squared_error(y_test, y_pred_auto),  mean_squared_error(y_train, poly_pipe.predict(X_train))


(7045.946151190245, 6450.839315158606)

**Practice: Getting results from make_pipeline**
- Obtain and report the coefficients, and the intercept.

In [11]:
poly_pipe.named_steps

{'polynomialfeatures': PolynomialFeatures(degree=3),
 'linearregression': LinearRegression()}

In [12]:
poly_pipe.named_steps['linearregression'].coef_

array([ 0.00000000e+00,  4.78463125e-02, -1.95203304e-06,  2.87628127e-11])

In [13]:
poly_pipe.named_steps['linearregression'].intercept_

169.33378283967744

In [14]:
poly_pipe.score(X_test, y_test), poly_pipe.score(X_train, y_train)

(0.3235362995692006, 0.3267372041364478)

### 1.3 Hyperparameter Tuning with Polynomial Regression
In the previous case, we consider a naive scenario where k = 3. Recall that k is the hyperparameter. To get a better prediction result, we may consider tune the degree k to find the k that gives the best performance.

To make best use of our data, and to avoid overfitting, we will apply cross-validation for performance measure. The best k should be chosen based on the (mean) performance of the validation set.


Sklearn provides a nice syntax that combines grid search and cross validation:
<br><div style="font-family:Calibri; text-align: center;"> **sklearn.model_selection.GridSearchCV(estimator, param_grid, scoring,  cv)** </div>

- estimator : the model. If a sequence of models, then use pipeline to put them together.
- param_grid : dictionary format. Specifies the potential choice of parameters. The keys must be correct.
- scoring : Performance measure. For linear models, default is R-square. Can be specified to mae (neg_mean_absolute_error), mse ('neg_mean_squared_error') as well.
- cv : Determines the cross-validation splitting strategy. If cv is integer (say, k), then k-fold cv. Default is 5-fold cv.


**Practice: Use GridSearchCV with polynomial regression - Train and Predict**

Apply grid search for hyperparameter tuning, and select the best model based on cross-validation performance. Use R-square as the performance measure. Potential candidate of hyperparameter: integers from 1 to 5, include 1 and 5.
- Define grid of hyperparameters.
- Define the estimator.
- Define gridsearchCV
- Train the model. What is the test mse? Is the model chosen based on the test mse?

In [15]:
# load gridsearchCV
from sklearn.model_selection import GridSearchCV

# Load other modules
from sklearn.preprocessing  import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

# S1. Define grid of parameter. 

param_poly = {'polynomialfeatures__degree' :  range(1, 6)  }   ## The name should be exactly the same,
                                                               ## otherwise cannot find which to specify.

# S2. Define estimator: use make_pipeline to combine two functions. 
grid_poly = PolynomialFeatures()
grid_lr = LinearRegression()

# Combines the polynomial feature and the linear regression model into a single pipeline.
grid_pipe = make_pipeline(grid_poly, grid_lr)  

# S3. Define GridSearchCV Estimation function, then train the model

gridsearch_poly = GridSearchCV(grid_pipe, param_poly, cv=5)
gridsearch_poly.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'polynomialfeatures__degree': range(1, 6)})

In [16]:
grid_pipe.get_params().keys()  # retrieve all the names of available parameters in pipeline 
                               # like 'polynomialfeatures__degree', 'linearregression'

dict_keys(['memory', 'steps', 'verbose', 'polynomialfeatures', 'linearregression', 'polynomialfeatures__degree', 'polynomialfeatures__include_bias', 'polynomialfeatures__interaction_only', 'polynomialfeatures__order', 'linearregression__copy_X', 'linearregression__fit_intercept', 'linearregression__n_jobs', 'linearregression__positive'])

In [17]:
gridsearch_poly.predict(X_test)

array([372.368752  , 336.66401182, 365.97019729, ..., 304.04738548,
       455.7230367 , 363.54271511])

**Outputs of GridSearchCV:**
- best_score_ : the mean validation score of the best model. The best model's performance measure, based on which the model is chosen.
- best_params_: the choice of hyperparameter
- best_estimator_: the best model of choice (and corresponding results). You can access model estimates from this attribute.
- cv_results_: detailed results stored (e.g., time & score of each hyperparameter, each iteration). Dictionary format.

**Practice: Collect Results from GridSearchCV**
- What is the chosen degree of polynomial regression?
- For the best model, report its performance score based on which the model is chosen.
- Explore attribute: cv_results_. Can you provide evidence of why the best model should be chosen? 
- Explore attribute:best_estimator_. Under the chosen model, what are the coefficients (include intercept)?

In [18]:
## 1. Chosen degree: this is the ...?
gridsearch_poly.best_params_

{'polynomialfeatures__degree': 3}

In [19]:
## 2. The performance is chosen based on ...?
gridsearch_poly.best_score_

0.31736242882259663

In [20]:
## 3. Check cv_results_
gridsearch_poly.cv_results_

{'mean_fit_time': array([0.0032445 , 0.00255961, 0.00251689, 0.00307822, 0.00219979]),
 'std_fit_time': array([0.00030609, 0.00046148, 0.00044986, 0.00025328, 0.00039988]),
 'mean_score_time': array([0.00140128, 0.00129828, 0.0010035 , 0.00109344, 0.0010416 ]),
 'std_score_time': array([4.91271710e-04, 3.24456682e-04, 5.24954183e-06, 1.33309752e-04,
        8.31127851e-05]),
 'param_polynomialfeatures__degree': masked_array(data=[1, 2, 3, 4, 5],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'polynomialfeatures__degree': 1},
  {'polynomialfeatures__degree': 2},
  {'polynomialfeatures__degree': 3},
  {'polynomialfeatures__degree': 4},
  {'polynomialfeatures__degree': 5}],
 'split0_test_score': array([0.28503762, 0.31455706, 0.32289995, 0.32421681, 0.32007107]),
 'split1_test_score': array([0.26901215, 0.30245627, 0.30871849, 0.30690419, 0.29903995]),
 'split2_test_score': array([0.30160033, 0.32395589, 0.33977123,

In [21]:
## 4. Check best_estimator_
gridsearch_poly.best_estimator_

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('linearregression', LinearRegression())])

## Ridge Regression

Estimating ridge regression is done by syntax:
<br> <center><span style="font-family:Calibri"> **sklearn.linear_model.Ridge()** </span></center>
The hyperparameter is specified as "alpha". By default, alpha = 1. *Note that this is the same for almost all models in sklearn.linear_model, including logistic regression*


**Practice**
- Prepare data as in polynomial case. Let $X$ be variables:  'GROSS_AREA', 'ROOMS', 'LIVING_AREA', 'LOT_SQFT', 'FLOORS', 'FULL_BATH'
- Run ridge regression without model selection or CV. Use defalt alpha = 1
- Run ridge regression with grid search and CV. Select tuning parameter from: [0.001, 0.01, 0.1, 1, 10,100].


In [22]:
# Data Loading and Splitting
var = ['GROSS_AREA', 'ROOMS', 'LIVING_AREA', 'LOT_SQFT', 'FLOORS', 'FULL_BATH']
X = house[var]
y = house['TOTAL_VALUE']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)


In [23]:
# Model and prediction
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha = 1)
ridge_reg.fit(X_train, y_train)

Ridge(alpha=1)

In [24]:
# Predict the price for X_test
y_test_pred_ridge = ridge_reg.predict(X_test)

In [25]:
# Report estimates  mse, mae, r^2
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, r2_score

mse(y_test, y_test_pred_ridge), mae(y_test, y_test_pred_ridge), r2_score(y_test, y_test_pred_ridge)

(2305.0161772171555, 35.787635972343594, 0.7787011510824884)

In [26]:
## Grid Search with CV
from sklearn.model_selection import GridSearchCV

# 1. Define a list of parameters (key is 'alpha')
param_ridge = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100] }

# 2. Define function and fit the data
grid_ridge = Ridge()
grid_ridge_est = GridSearchCV(grid_ridge, param_ridge, cv=5)
grid_ridge_est.fit(X_train, y_train)

# 3.1 Present performance measure
print(grid_ridge_est.best_score_)

# 3.2 find best hyperparameters
print(grid_ridge_est.best_params_)

# 3.3 find best parameter estimates
print(grid_ridge_est.score(X_test, y_test))


0.783214249774319
{'alpha': 1}
0.7787011510824884


## LASSO

Estimating LASSO regression is done by creating a Lasso object using syntax:
<br> <center><span style="font-family:Calibri"> **sklearn.linear_model.Lasso** </span></center>
- hyperparameter is also alpha. Default is 1.

**Practice**
- Prepare data as in polynomial case. Let $X$ be 'GROSS_AREA', 'ROOMS', 'LIVING_AREA', 'LOT_SQFT', 'FLOORS', 'FULL_BATH'.
- Run LASSO regression without model selection or CV
- Run LASSO regression with grid search and CV. Select tuning parameter from: [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10].

In [27]:
# Model and prediction with default hyperparameter
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=1)
lasso_reg.fit(X_train, y_train)

y_test_lasso = lasso_reg.predict(X_test)
y_test_lasso

array([314.33196863, 307.03292635, 300.07618422, ..., 319.39027686,
       550.08202434, 353.05533476])

In [28]:
# Grid Search with CV - LASSO Case

# 1. Define a list of parameters (key is 'alpha')
param_lasso = {'alpha': [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100] }

# 2. Define function and fit the data
grid_lasso = Lasso()
grid_lasso_est = GridSearchCV(grid_lasso, param_lasso, cv=5)
grid_lasso_est.fit(X_train, y_train)

# 3.1 Present performance measure
print(grid_lasso_est.best_score_)

# 3.2 find best hyperparameters
print(grid_lasso_est.best_params_)

# 3.3 find best parameter estimates
print(grid_lasso_est.score(X_test, y_test))

0.7832136126251984
{'alpha': 0}
0.7787051868085179
